In [1]:
from scipy import ndimage
import numpy as np

In [2]:
def imregionalmax(image, footprint):
    """Find the regional max of an ND image. An approximation of MATLAB's
    imregionalmax function. Result only differs when surrounding pixels
    have the same value as the center.

    Parameters:
    - image: the input image
    - footprint: a boolean ndarray specifying which neighboring pixels should be considered
                 for thresholding, see scipy.ndimage.generate_binary_structure.
    Returns:
    - a bitmask image, where '1' indicates local maxima.
    Author:
    - Yu Fang
    References:
    - https://github.com/bhardwajvijay/Utils/blob/master/utils.cpp
    - https://stackoverflow.com/questions/5550290/find-local-maxima-in-grayscale-image-using-opencv
    """
    # dialate the image so that small values are replaced by local max
    local_max = ndimage.grey_dilation(image, footprint=footprint, mode='reflect')
    # non-local max pixels (excluding pixel w/ constant 3x3 neighborhood)
    # will be replaced by local max, so the values will increase. remove them.
    # so the result is either local max or constant neighborhood.
    max_mask = image >= local_max
    # erode the image so that high values are replaced by local min
    local_min = ndimage.grey_erosion(image, footprint=footprint, mode='reflect')
    # only local min pixels and pixels w/ constant 3x3 neighborhood
    # will stay the same, otherwise pixels will be replaced by the local
    # min and become smaller. We only take non-local min, non-constant values.
    min_mask = image > local_min
    # boolean logic hack
    #   (local max || constant) && (!local min && !constant)
    # = local max && !local min && !constant
    # = local max && !constant
    return (max_mask & min_mask).astype(np.uint8)

In [3]:
def imregionalmin(image, footprint):
    """Find the regional min of an ND image. An approximation of MATLAB's
    imregionalmin function. Result only differs when surrounding pixels
    have the same value as the center.

    Parameters:
    - image: the input image
    - footprint: a boolean ndarray specifying which neighboring pixels should be considered
                 for thresholding, see scipy.ndimage.generate_binary_structure.
    Returns:
    - a bitmask image, where '1' indicates local maxima.
    Author:
    - Yu Fang
    References:
    - https://github.com/bhardwajvijay/Utils/blob/master/utils.cpp
    - https://stackoverflow.com/questions/5550290/find-local-maxima-in-grayscale-image-using-opencv
    """
    # erode the image so that high values are replaced by local min
    local_min = ndimage.grey_erosion(image, footprint=footprint, mode='reflect')
    # non-local min pixels (excluding pixel w/ constant 3x3 neighborhood)
    # will be replaced by local min, so the values will decrease. remove them.
    # so the result is either local min or constant neighborhood.
    min_mask = image <= local_min
    # dialate the image so that small values are replaced by local max
    local_max = ndimage.grey_dilation(image, footprint=footprint, mode='reflect')
    # only local max pixels and pixels w/ constant 3x3 neighborhood
    # will stay the same, otherwise pixels will be replaced by the local
    # max and become larger. We only take non-local max, non-constant values.
    max_mask = image < local_max
    # boolean logic hack
    #   (local min || constant) && (!local max && !constant)
    # = local min && !local max && !constant
    # = local min && !constant
    return (max_mask & min_mask).astype(np.uint8)

In [4]:
# imregionalmax example 2D (remove this block in the final submission)
A = 10 * np.ones((10, 10))
A[1:4, 1:4] = 22
A[5:8, 5:8] = 33
A[1, 7] = 44
A[2, 8] = 45
A[3, 9] = 44
print(f"A:\n{A}")
print(f"regionmal max:\n{imregionalmax(A, ndimage.generate_binary_structure(2,2))}")

A:
[[10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
 [10. 22. 22. 22. 10. 10. 10. 44. 10. 10.]
 [10. 22. 22. 22. 10. 10. 10. 10. 45. 10.]
 [10. 22. 22. 22. 10. 10. 10. 10. 10. 44.]
 [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10. 33. 33. 33. 10. 10.]
 [10. 10. 10. 10. 10. 33. 33. 33. 10. 10.]
 [10. 10. 10. 10. 10. 33. 33. 33. 10. 10.]
 [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]]
regionmal max:
[[0 0 0 0 0 0 0 0 0 0]
 [0 1 1 1 0 0 0 0 0 0]
 [0 1 0 1 0 0 0 0 1 0]
 [0 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 0 0]
 [0 0 0 0 0 1 0 1 0 0]
 [0 0 0 0 0 1 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [5]:
# imregionalmax example 3D (remove this block in the final submission)
A = 10 * np.ones((3,10, 10))
A[1, 1:4, 1:4] = 22
A[0, 1:4, 1:4] = 22
A[2, 1:4, 1:4] = 22
A[2, 5, 9] = 33
A[1, 2, 2] = 33
A[1, 5:8, 5:8] = 33
A[1, 1, 7] = 44
A[1, 2, 8] = 45
A[2, 2, 8] = 46
A[1, 3, 9] = 44
print(f"A:\n{A}")
print(f"regionmal max:\n{imregionalmax(A, ndimage.generate_binary_structure(3,3))}")

A:
[[[10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
  [10. 22. 22. 22. 10. 10. 10. 10. 10. 10.]
  [10. 22. 22. 22. 10. 10. 10. 10. 10. 10.]
  [10. 22. 22. 22. 10. 10. 10. 10. 10. 10.]
  [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
  [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
  [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
  [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
  [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
  [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]]

 [[10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
  [10. 22. 22. 22. 10. 10. 10. 44. 10. 10.]
  [10. 22. 33. 22. 10. 10. 10. 10. 45. 10.]
  [10. 22. 22. 22. 10. 10. 10. 10. 10. 44.]
  [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
  [10. 10. 10. 10. 10. 33. 33. 33. 10. 10.]
  [10. 10. 10. 10. 10. 33. 33. 33. 10. 10.]
  [10. 10. 10. 10. 10. 33. 33. 33. 10. 10.]
  [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
  [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]]

 [[10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
  [10. 22. 22. 22. 10. 10. 10. 10. 10. 10.]
  [10. 22. 22. 22. 10. 10

In [6]:
# imregionalmin example 2D (remove this block in the final submission)
A = 100 * np.ones((10, 10))
A[1:4, 1:4] = 22
A[5:8, 5:8] = 33
A[1, 7] = 44
A[2, 8] = 9
A[3, 9] = 44
print(f"A:\n{A}")
print(f"regionmal min:\n{imregionalmin(A, ndimage.generate_binary_structure(2,2))}")

A:
[[100. 100. 100. 100. 100. 100. 100. 100. 100. 100.]
 [100.  22.  22.  22. 100. 100. 100.  44. 100. 100.]
 [100.  22.  22.  22. 100. 100. 100. 100.   9. 100.]
 [100.  22.  22.  22. 100. 100. 100. 100. 100.  44.]
 [100. 100. 100. 100. 100. 100. 100. 100. 100. 100.]
 [100. 100. 100. 100. 100.  33.  33.  33. 100. 100.]
 [100. 100. 100. 100. 100.  33.  33.  33. 100. 100.]
 [100. 100. 100. 100. 100.  33.  33.  33. 100. 100.]
 [100. 100. 100. 100. 100. 100. 100. 100. 100. 100.]
 [100. 100. 100. 100. 100. 100. 100. 100. 100. 100.]]
regionmal min:
[[0 0 0 0 0 0 0 0 0 0]
 [0 1 1 1 0 0 0 0 0 0]
 [0 1 0 1 0 0 0 0 1 0]
 [0 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 0 0]
 [0 0 0 0 0 1 0 1 0 0]
 [0 0 0 0 0 1 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
